# IT4065C — In‑Class Lab Notebook  
## Metadata, Classification Tags, and Data Quality (Server Metrics Dataset)

**How to use this notebook (student-friendly):**
- Run the notebook **top to bottom**, one cell at a time.
- Cells are labeled to match the lab steps (Step 1, Step 2, …).
- The notebook handles computations; your job is to **interpret results** and fill the Word template.

> Tip: You are not graded on Python. You are graded on the **accuracy of your metadata** and the **quality of your governance reasoning**.


## Step 0 — Setup
This cell imports libraries and sets display options.  
Run it once at the beginning.

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 140)
pd.set_option("display.max_colwidth", 80)


## Step 1 — Load the dataset
This loads **M1_data_for_metadata.csv** into a pandas DataFrame named `df`.

In [ ]:
# If this notebook is in the same folder as the CSV, this will work as-is.
# If needed, change the path below (e.g., 'data/M1_data_for_metadata.csv')
CSV_PATH = "M1_data_for_metadata.csv"

df = pd.read_csv(CSV_PATH)

df.head()


## Step 2 — Dataset overview (for your template)
This gives you the number of rows/columns and a quick summary you can use in **Dataset Overview**.

In [ ]:
rows, cols = df.shape
print(f"Rows: {rows:,}")
print(f"Columns: {cols:,}")
print("\nColumn names:")
for c in df.columns:
    print(f" - {c}")

print("\nData types (pandas):")
display(df.dtypes.to_frame("dtype"))


## Step 3 — Sample data (first 8 rows)
Use this output to take a screenshot for the **Sample Data** section of the Word template.

In [ ]:
df.head(8)

## Step 4 — Column-level metadata table (auto-generated)
This creates a table with:
- Column Name
- Data Type (numeric / categorical / text)
- Missing Values (count and %)
- Example Values (up to 3)

You can **copy/paste** this into your Word template, or export it using the next cell.

In [ ]:
def classify_dtype(series: pd.Series) -> str:
    if pd.api.types.is_numeric_dtype(series):
        return "Numeric"
    # treat low-cardinality strings as categorical
    if pd.api.types.is_string_dtype(series) or pd.api.types.is_object_dtype(series):
        nunique = series.dropna().nunique()
        if nunique <= 50:
            return "Categorical"
        return "Text"
    return "Other"

def example_values(series: pd.Series, k: int = 3):
    vals = series.dropna().unique()
    vals = vals[:k]
    # convert numpy types to python types for readability
    return [str(v) for v in vals]

rows = len(df)
meta_rows = []
for col in df.columns:
    s = df[col]
    missing_count = int(s.isna().sum())
    missing_pct = (missing_count / rows) * 100
    meta_rows.append({
        "Column Name": col,
        "Description (you write in Word)": "",
        "Data Type": classify_dtype(s),
        "Missing Values (count)": missing_count,
        "Missing Values (%)": round(missing_pct, 1),
        "Example Values (up to 3)": ", ".join(example_values(s, 3))
    })

metadata_table = pd.DataFrame(meta_rows)
metadata_table


### Optional: Export the metadata table
This exports the auto-generated table to a CSV file you can open in Excel, then copy into Word.

In [ ]:
OUTPUT_META_CSV = "metadata_table_generated.csv"
metadata_table.to_csv(OUTPUT_META_CSV, index=False)
print(f"Saved: {OUTPUT_META_CSV}")

## Step 5 — Data quality checks (missingness, duplicates, ranges)
Use this section to write **4.1 Overall Data Completeness & Key Issues** and **4.2 Recommendations**.

In [ ]:
# Missingness profile
missing = df.isna().sum().to_frame("missing_count")
missing["missing_%"] = (missing["missing_count"] / len(df) * 100).round(1)
missing = missing.sort_values("missing_count", ascending=False)

print("Missingness (highest first):")
display(missing)

# Quick single-sentence summary helper
top_missing = missing[missing["missing_count"] > 0].head(3)
if len(top_missing) == 0:
    print("No missing values detected.")
else:
    cols = ", ".join(top_missing.index.tolist())
    print(f"Top columns with missing values: {cols}")


In [ ]:
# Duplicate row check
dup_count = int(df.duplicated().sum())
print(f"Duplicate rows: {dup_count}")

# ID uniqueness check (common governance expectation)
if "ID" in df.columns:
    id_dups = int(df["ID"].duplicated().sum())
    print(f"Duplicate IDs: {id_dups}")


### Numeric summary (useful for spotting outliers)
This helps you identify unusual ranges that could indicate data quality issues.

In [ ]:
numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
display(df[numeric_cols].describe().T)

print("\nTip: If you see extreme min/max values compared to typical values (25%/50%/75%), mention that as a potential outlier issue.")


### Simple outlier flags (IQR rule)
This does not 'prove' something is wrong, but it highlights values that deserve attention.

In [ ]:
def iqr_outlier_count(series: pd.Series) -> int:
    s = series.dropna()
    if len(s) < 10:
        return 0
    q1, q3 = np.percentile(s, [25, 75])
    iqr = q3 - q1
    lo = q1 - 1.5 * iqr
    hi = q3 + 1.5 * iqr
    return int(((s < lo) | (s > hi)).sum())

outlier_summary = []
for c in numeric_cols:
    outlier_summary.append({"column": c, "iqr_outliers": iqr_outlier_count(df[c])})

outlier_summary = pd.DataFrame(outlier_summary).sort_values("iqr_outliers", ascending=False)
display(outlier_summary)

print("\nUse: If a column has many outliers, recommend validation rules or scaling/normalization depending on the use case.")


### Categorical checks (consistency and value counts)
Useful for catching inconsistent labels (e.g., 'Fiber' vs 'FIBER' vs 'fiber').

In [ ]:
categorical_cols = [c for c in df.columns if metadata_table.set_index("Column Name").loc[c, "Data Type"] == "Categorical"]

for c in categorical_cols:
    print(f"\n=== {c} ===")
    vc = df[c].astype("string").str.strip().value_counts(dropna=False)
    display(vc.head(15))

print("\nTip: If you see messy categories (extra spaces, inconsistent casing), recommend standardizing values.")


## Step 6 — Classification tags and stewardship (write-ups)
This section helps you draft the **Classification and Stewardship** part of your report.

Fill in the answers in the next cell (short phrases are fine), then run it to produce a clean paragraph you can paste into Word.

In [ ]:
# ---- Student fill-in section ----
STRUCTURAL_CLASS = "Structured"          # Structured / Semi-Structured / Unstructured
SENSITIVITY_CLASS = "Internal"           # Public / Internal / Confidential / Restricted
REGULATORY_RELEVANCE = "None"            # GDPR / HIPAA / PCI / None / Uncertain

STRUCTURAL_JUST = "Tabular CSV with clearly defined columns and data types."
SENSITIVITY_JUST = "Operational infrastructure metrics could reveal performance or security posture if widely shared."
REGULATORY_JUST = "No direct PII/PHI/PCI fields appear in the dataset, but internal security policy still applies."

DATA_OWNER_ROLE = "IT Infrastructure Manager"
DATA_STEWARD_ROLE = "Systems Operations Analyst"

OWNER_RESP_1 = "Approves access decisions and intended uses."
OWNER_RESP_2 = "Sets retention and sharing policy for the dataset."

STEWARD_RESP_1 = "Maintains metadata accuracy and classification tags."
STEWARD_RESP_2 = "Monitors data quality and coordinates fixes with engineering teams."
# --------------------------------

print("### Classification (paste into Word)")
print(f"Structural classification: {STRUCTURAL_CLASS} — {STRUCTURAL_JUST}")
print(f"Sensitivity classification: {SENSITIVITY_CLASS} — {SENSITIVITY_JUST}")
print(f"Regulatory relevance: {REGULATORY_RELEVANCE} — {REGULATORY_JUST}")

print("\n### Stewardship (paste into Word)")
print(f"Data Owner (role): {DATA_OWNER_ROLE}. Responsibilities: (1) {OWNER_RESP_1} (2) {OWNER_RESP_2}")
print(f"Data Steward (role): {DATA_STEWARD_ROLE}. Responsibilities: (1) {STEWARD_RESP_1} (2) {STEWARD_RESP_2}")


## Step 7 — Draft helper for Section 4.1 (Template paragraph)
This cell produces a short draft paragraph for **4.1 Overall Data Completeness & Key Issues** using placeholders you can edit.

In [ ]:
# Auto-pick the top missing columns to help you draft 4.1
missing_sorted = df.isna().sum().sort_values(ascending=False)
top = [c for c in missing_sorted.index if missing_sorted[c] > 0][:3]

# Identify numeric vs categorical example columns
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
cat_cols = [c for c in df.columns if not pd.api.types.is_numeric_dtype(df[c])]

num_ex = num_cols[:2]
cat_ex = cat_cols[:2]

print("Copy/edit this paragraph into your Word template (Section 4.1):\n")
print(f"Overall, the dataset contains moderate data completeness. Significant missing values are observed in {', '.join(top) if top else '[no major columns]'}, which may affect performance monitoring and downstream analysis. "
      f"The dataset includes a mix of quantitative attributes (e.g., {', '.join(num_ex)}) and categorical attributes (e.g., {', '.join(cat_ex)}). "
      "From a data governance perspective, these quality issues could reduce reliability of operational decisions if not addressed.")


## Step 8 — What to copy into the Word template (quick checklist)

From this notebook, you can copy:
- Step 2: row/column counts + data types
- Step 3: screenshot of first 8 rows
- Step 4: metadata table (and missingness counts)
- Step 5: quality issues (missingness, outliers, category consistency)
- Step 6: classification & stewardship write-up
- Step 7: draft paragraph for Section 4.1

You still need to write:
- Column descriptions (plain language)
- 2 quality recommendations (with governance justification)
- 2 practical use cases
